<a href="https://colab.research.google.com/github/tjunjet/til_nothing/blob/wh-nlp/aris_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install tensorboardx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
import os
from PIL import Image
from scipy.fftpack import fft
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
path = '/content/gdrive/MyDrive/TIL/Qualifiers/NLP Training Dataset/ASR Training Dataset'

In [6]:
filepath = '/content/gdrive/MyDrive/TIL/Qualifiers/NLP Training Dataset/ASR Training Dataset/angry/00137a41c0.wav'

Loading files

In [7]:
import os
import chardet

def file_search(dirname, ret, list_avoid_dir=[]):
    filenames = os.listdir(dirname)
    
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)

        if os.path.isdir(full_filename) :
            if full_filename.split('/')[-1] in list_avoid_dir:
                continue
            else:
                file_search(full_filename, ret, list_avoid_dir)
            
        else:
            ret.append( full_filename ) 

In [8]:
#list_files = []
file_dict = dict()
for name in ['angry','fear','happy','neutral','sad']:
    folder = path + name
    list_files = []
    file_search(path, list_files)
    list_files = sorted(list_files)
    file_dict[name] = list_files
    print (name + ", #sum files: " + str(len(list_files)))
#extract_feature( list_files, out_file )


angry, #sum files: 3500
fear, #sum files: 3500
happy, #sum files: 3500
neutral, #sum files: 3500
sad, #sum files: 3500


Helper Functions

In [9]:
def audio2spectrogram(filepath):
    fig = plt.figure(figsize=(5,5))
    samplerate, test_sound  = wavfile.read(filepath,mmap=True)
    print('samplerate',samplerate)
    _, spectrogram = log_specgram(test_sound, samplerate)
    print(spectrogram.shape)
    print(type(spectrogram))
    plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    return spectrogram

def audio2wave(filepath):
    fig = plt.figure(figsize=(5,5))
    samplerate, test_sound  = wavfile.read(filepath,mmap=True)
    plt.plot(test_sound)

In [10]:
def log_specgram(audio, sample_rate, window_size=40,
                 step_size=20, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    #print('noverlap',noverlap)
    #print('nperseg',nperseg)
    freqs, _, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, np.log(spec.T.astype(np.float32) + eps)

In [11]:
N_CHANNELS = 3
def get_3d_spec(Sxx_in, moments=None):
    if moments is not None:
        (base_mean, base_std, delta_mean, delta_std,
             delta2_mean, delta2_std) = moments
    else:
        base_mean, delta_mean, delta2_mean = (0, 0, 0)
        base_std, delta_std, delta2_std = (1, 1, 1)
    h, w = Sxx_in.shape
    right1 = np.concatenate([Sxx_in[:, 0].reshape((h, -1)), Sxx_in], axis=1)[:, :-1]
    delta = (Sxx_in - right1)[:, 1:]
    delta_pad = delta[:, 0].reshape((h, -1))
    delta = np.concatenate([delta_pad, delta], axis=1)
    right2 = np.concatenate([delta[:, 0].reshape((h, -1)), delta], axis=1)[:, :-1]
    delta2 = (delta - right2)[:, 1:]
    delta2_pad = delta2[:, 0].reshape((h, -1))
    delta2 = np.concatenate([delta2_pad, delta2], axis=1)
    base = (Sxx_in - base_mean) / base_std
    delta = (delta - delta_mean) / delta_std
    delta2 = (delta2 - delta2_mean) / delta2_std
    stacked = [arr.reshape((h, w, 1)) for arr in (base, delta, delta2)]
    return np.concatenate(stacked, axis=2)

EXTACT THE MFCC FEATURE USING LIBROSA



In [ ]:
no_rows=len(list_files)*5
index=0
sprectrogram_shape=[]
docs = []
bookmark=0
extraLabel=0
for key in ['angry','fear','happy','neutral','sad']:
  count=0
  for everyFile in file_dict[name]:
    if count < 100:
      if(everyFile.split('/')[-1].endswith('.wav')):
        filename=everyFile.split('/')[-1].strip('.wav')
        lable=name
        print('label',lable)
        #sprectrogram_shape.append(audio2spectrogram(everyFile))
        spector=audio2spectrogram(everyFile)
        spector=get_3d_spec(spector)
        npimg = np.transpose(spector,(2,0,1))
        input_tensor=torch.tensor(npimg)
        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
        #X, sample_rate = librosa.load(everyFile, res_type='kaiser_fast',sr=22050*2)
        #sample_rate = np.array(sample_rate)
        #mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=13),axis=0)
        #feature = mfccs
        docs.append({
          'fileName':everyFile.split('/')[-1].strip('.wav'),
          #'feature_mfcc':feature,
          'sprectrome':input_batch,
          'label':lable
                })
        index+=1
        print('index',index)

import json
json.dumps('content/audio-info.json')
    

label sad
samplerate 16000
(103, 321)
<class 'numpy.ndarray'>
index 1
label sad
samplerate 16000
(107, 321)
<class 'numpy.ndarray'>
index 2
label sad
samplerate 16000
(104, 321)
<class 'numpy.ndarray'>
index 3
label sad
samplerate 16000
(103, 321)
<class 'numpy.ndarray'>
index 4
label sad
samplerate 16000
(73, 321)
<class 'numpy.ndarray'>
index 5
label sad
samplerate 16000
(103, 321)
<class 'numpy.ndarray'>
index 6
label sad
samplerate 16000
(113, 321)
<class 'numpy.ndarray'>
index 7
label sad
samplerate 16000
(102, 321)
<class 'numpy.ndarray'>
index 8
label sad
samplerate 16000
(106, 321)
<class 'numpy.ndarray'>
index 9
label sad
samplerate 16000
(95, 321)
<class 'numpy.ndarray'>
index 10
label sad
samplerate 16000
(117, 321)
<class 'numpy.ndarray'>
index 11
label sad
samplerate 16000
(121, 321)
<class 'numpy.ndarray'>
index 12
label sad
samplerate 16000
(107, 321)
<class 'numpy.ndarray'>
index 13
label sad
samplerate 16000
(109, 321)
<class 'numpy.ndarray'>
index 14
label sad
sampler

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Streaming output truncated to the last 5000 lines.
(111, 321)
<class 'numpy.ndarray'>
index 564
label sad
samplerate 16000
(129, 321)
<class 'numpy.ndarray'>
index 565
label sad
samplerate 16000
(105, 321)
<class 'numpy.ndarray'>
index 566
label sad
samplerate 16000
(149, 321)
<class 'numpy.ndarray'>
index 567
label sad
samplerate 16000
(116, 321)
<class 'numpy.ndarray'>
index 568
label sad
samplerate 16000
(112, 321)
<class 'numpy.ndarray'>
index 569
label sad
samplerate 16000
(115, 321)
<class 'numpy.ndarray'>
index 570
label sad
samplerate 16000
(113, 321)
<class 'numpy.ndarray'>
index 571
label sad
samplerate 16000
(99, 321)
<class 'numpy.ndarray'>
index 572
label sad
samplerate 16000
(86, 321)
<class 'numpy.ndarray'>
index 573
label sad
samplerate 16000
(125, 321)
<class 'numpy.ndarray'>
index 574
label sad
samplerate 16000
(110, 321)
<class 'numpy.ndarray'>
index 575
label sad
samplerate 16000
(105, 321)
<class 'numpy.ndarray'>
index 576
label sad
samplerate 16000
(104, 321)
<cla

Test AlexNet

In [ ]:
import torch
import torch.nn as nn
#from .utils import load_state_dict_from_url
from torch.hub import load_state_dict_from_url

__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((12, 12))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        print('features',x.shape)
        
        #x = self.avgpool(x)
        #print('avgpool',x.shape)
        #x = torch.flatten(x, 1)
        #print('flatten',x.shape)
        #x = self.classifier(x)
        return x
def alexnet(pretrained=False, progress=True, **kwargs):
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

Modified AlexNet

In [ ]:
class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=5):
        super(ModifiedAlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.features(x)
        #print('features',x.shape)
        x=torch.flatten(x, start_dim=2)#a1,a2,a3......al{a of dim c} 
        x=torch.sum(x, dim=2)#a1*alpha1+a2*alpha2+.......+al*alphal
        #print(x.shape)
        x=self.classifier(x)
        #print('classifier',x)
        #x=self.softmax(x)
        #print('softmax',x)
        #x = self.avgpool(x)
        #print('avgpool',x.shape)
        #x = torch.flatten(x, 1)
        #print('flatten',x.shape)
        #x = self.classifier(x)
        return x
   
def modifiedAlexNet(pretrained=False, progress=True, **kwargs):
    model_modified = ModifiedAlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model_modified.load_state_dict(state_dict)
    return model_modified

In [ ]:
original_model=alexnet(pretrained=True)
original_dict = original_model.state_dict()
modifiedAlexNet=modifiedAlexNet(pretrained=False)
modified_model_dict = modifiedAlexNet.state_dict()
pretrained_modified_model_dict = {k: v for k, v in original_dict.items() if k in modified_model_dict}
modifiedAlexNet.to('cuda')

In [ ]:
x=[]
for key in file_dict:
  x+=audio2spectrogram(file_dict[key])
x=get_3d_spec(x)
npimg = np.transpose(x,(2,0,1))
input_tensor=torch.tensor(npimg)

input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    modifiedAlexNet.to('cuda')
with torch.no_grad():
    output = modifiedAlexNet(input_batch)
    #output.squeeze().shape
    #output=torch.flatten(output, start_dim=2)
    #print(output.shape)
    #output=torch.sum(output, dim=2)
    print(output)

Input data shuffling 

In [ ]:
import json 
import random

# docs = json.loads('content/audio-info.json')
random.shuffle(docs)
random.shuffle(docs)
random.shuffle(docs)
total_length=len(docs)
train_length=int(.9*total_length)
train_list=docs[0:train_length]
test_list=docs[train_length:]
print('no of items for train ',len(train_list))
print('no of items for test ',len(test_list))

Plot training loss and accuracy

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./

Model Params

In [ ]:
for name, param in modifiedAlexNet.named_parameters():
      if(param.requires_grad):
        print(name)
      else:
        print('no grad',name)

Optimizer

In [ ]:
import torch.optim as optim
from transformers import AdamW
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(modifiedAlexNet.parameters(),
                  lr =  2e-4, 
                  eps = 1e-8
                )
from transformers import get_linear_schedule_with_warmup

NUM_EPOCHS=16

writer = SummaryWriter(log_dir='/content/')
total_steps = len(train_list) * NUM_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Training Loop

In [ ]:
total_steps = 1


seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in range(NUM_EPOCHS):
  modifiedAlexNet.train()
  for every_trainlist in train_list:
    label1=every_trainlist['label']
    label1=torch.tensor([label1])
    sprectrome=every_trainlist['sprectrome']
    if(sprectrome.shape[2]>65):
      optimizer.zero_grad()
      sprectrome = sprectrome.to('cuda')
      label1=label1.to('cuda')
      modifiedAlexNet.zero_grad()
      output = modifiedAlexNet(sprectrome)
      print('softmax output ',output)
      loss = criterion(output, label1)
      print('label1',label1)
      print('loss',loss.item())
      loss.backward()
      torch.nn.utils.clip_grad_norm_(modifiedAlexNet.parameters(), 1.0)
      optimizer.step()
      scheduler.step()
      _, preds = torch.max(output, 1)
      accuracy = torch.sum(preds == label1)
      print('accuracy.item()',accuracy.item())
      print('preds',preds)
      if total_steps % 10 == 0:
        with torch.no_grad():
          _, preds = torch.max(output, 1)
          accuracy = torch.sum(preds == label1)
          print('Epoch: {} \tStep: {} \tLoss: {:.4f} \tAcc: {}'.format(epoch + 1, total_steps, loss.item(), accuracy.item()))
          writer.add_scalar('loss', loss.item(), total_steps)
          writer.add_scalar('accuracy', accuracy.item(), total_steps)                     
      total_steps+=1